In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pprint import pprint

In [2]:
df = pd.read_json("contacts.json")
nd_array = df.values
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,


In [3]:
df.describe()

,Id,Contacts
count,500000.000000,500000.000000
mean,249999.500000,1.996094
std,144337.711635,1.412825
min,0.000000,0.000000
25%,124999.750000,1.000000
50%,249999.500000,2.000000
75%,374999.250000,3.000000
max,499999.000000,4.000000


In [4]:
ids = df["Id"].values.tolist()
len(ids)

500000

In [5]:
df.loc[18]

Id                                 18
Email         qOkuzhOuPsbltatw@qq.com
Phone                                
Contacts                            4
OrderId     zSekjOkRLyiGjvOPikTKwksFK
Name: 18, dtype: object

In [6]:
df.loc[df["OrderId"] == "zSekjOkRLyiGjvOPikTKwksFK"]

,Id,Email,Phone,Contacts,OrderId
18,18,qOkuzhOuPsbltatw@qq.com,,4,zSekjOkRLyiGjvOPikTKwksFK
274269,274269,tndZkvBDwg@gmail.com,66873440472,0,zSekjOkRLyiGjvOPikTKwksFK
327057,327057,sjoEThSNaVBlYk@gmail.com,,1,zSekjOkRLyiGjvOPikTKwksFK


In [7]:
df.loc[df["Phone"] == "66873440472"]

,Id,Email,Phone,Contacts,OrderId
26580,26580,pQLTiUynFTMQGlQvETVv@hotmail.com,66873440472,3,vQXjSpqgUekYYuSeVOpdaPZme
70474,70474,TpnGzfIuAzcV@hotmail.com,66873440472,2,
274269,274269,tndZkvBDwg@gmail.com,66873440472,0,zSekjOkRLyiGjvOPikTKwksFK


### Get Relation

In [8]:
main_memory = {}
user_ids = set()
output = []

## Using numpy array

In [12]:
def np_get_row_relation(row, user_ids, memory):
    current_user_id = row[0]

    if current_user_id in main_memory:
        return main_memory[current_user_id]

    if current_user_id not in memory:
        memory.append(current_user_id)
        
        order_id = row[4]
        email = row[1]
        phone = row[2]
        
        user_ids.add(current_user_id)
        
        if order_id != "":
            orderid_link = nd_array[(nd_array[:, 4] == order_id) & (nd_array[:, 0] == current_user_id)]
            user_ids = user_ids.union(orderid_link[:, 0])
            
            for orderid_row in orderid_link:
                user_ids = user_ids.union(np_get_row_relation(orderid_row, user_ids, memory))
            
        if email != "":
            email_link = nd_array[(nd_array[:, 1] == email) & (nd_array[:, 0] == current_user_id)] 
            user_ids = user_ids.union(email_link[:, 0])
            
            for email_row in email_link:
                user_ids = user_ids.union(np_get_row_relation(email_row, user_ids, memory))

        if phone != "":
            phone_link = nd_array[(nd_array[:, 2] == phone) & (nd_array[:, 0] == current_user_id)]
            user_ids = user_ids.union(phone_link[:, 0])
            
            for phone_row in phone_link:
                user_ids = user_ids.union(np_get_row_relation(phone_row, user_ids, memory))

    return user_ids

def numpy_solution():
    output = []
    for row in tqdm(nd_array):
        current_user_id = row[0]
        
        if current_user_id in main_memory:
            user_ids = main_memory[current_user_id]
            output.append({"ticket_id": current_user_id, "ticket_trace/contact": user_ids})
            continue

        user_ids = {current_user_id}

        # No relation
        if row[4] == "" and row[1] == "" and row[2] == "":
            output.append({"ticket_id": current_user_id, "ticket_trace/contact": user_ids})
            continue

        user_ids = user_ids.union(np_get_row_relation(row, user_ids, []))

        for uid in user_ids:
            main_memory[uid] = user_ids

        output.append({"ticket_id": current_user_id, "ticket_trace/contact": user_ids})
        
numpy_solution()

100%|█████████████████████████████████████████████| 500000/500000 [3:27:53<00:00, 40.08it/s]


### Sum Contacts & Submission
```
ticket_id,ticket_trace/contact
0,"0, 1"
1,"1, 2"
2,"2-150, 6"
3,"3-100, 3"
```


In [ ]:
header = "ticket_id,ticket_trace/contact"
with open("submission.csv", "w+") as f:
    f.write(header)
    for entry in tqdm(output):
        contacts = df.iloc[entry['trace']]["Contacts"].sum().tolist()
        trace = "-".join([str(trace) for trace in sorted(entry['trace'])])
        syntax = "{},\"{},{}\"\n".format(entry["id"], trace, contacts)
        f.write(syntax)

In [ ]:
pprint(output)

### Submission Syntax
```
ticket_id,ticket_trace/contact
0,"0, 1"
1,"1, 2"
2,"2-150, 6"
3,"3-100, 3"
```